# Dense Matrix-Vector Product, 2D Decomposition

This assignment will be graded on four nodes with 28 cores, but for development you can run anywhere.

**(Coding, 6 pt):** In this assignment you will complete the dense matrix vector product code where the matrix is decomposed by a 2D decomposition.

In [1]:
module unload cse6230
module load cse6230/gcc-omp-gpu

|                                                                         |
|       A note about python/3.6:                                          |
|       PACE is lacking the staff to install all of the python 3          |
|       modules, but we do maintain an anaconda distribution for          |
|       both python 2 and python 3. As conda significantly reduces        |
|       the overhead with package management, we would much prefer        |
|       to maintain python 3 through anaconda.                            |
|                                                                         |
|       All pace installed modules are visible via the module avail       |
|       command.                                                          |
|                                                                         |


In [2]:
make clean
make dense_matrix_vector_product

rm -f *.o dense_matrix_vector_product
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dense_matrix_vector_product.o dense_matrix_vector_product.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_args.o dmv_args.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv.o dmv.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_global_size.o dmv_global_size.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_layout.o dmv_layout.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_matvec_2d.o dmv_matvec_2d.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_matvec.o dmv_matvec.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_matvec_col.o dmv_matvec_col.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 

The `dense_matrix_vector_product` program computes distributed dense matrix vector products by a few different distributions and algorithms that you can choose from:

In [3]:
mpirun -f ${PBS_NODEFILE} -np 1 ./dense_matrix_vector_product -h

Usage: ./dense_matrix_vector_product [-s scale] [-e seed] [-v verbosity] [-d debug]
          [-g global_size_strategy] [-l layout_strategy]
          [-p matrix_partition_strategy] [-m matvec_strategy]

global_size_strategy:tree_subcomm, tree_recurse, reduce_bcast, allreduce
layout_strategy: gather, allgather, scan
matrix_partition_strategy: rows, cols, 2d
matvec_strategy: ssend, issend, allgathervRight vector norm: 18.6581
Matrix Frobenius norm: 596.735
[0] Average time per matvec: 0.00102035
Left vector norm: 318.194


The vector and matrix entries are computed from a random hash of their global indices, meaning that they should be the same regardless of the layout or the number of processes.  A quick check on this is to compute the norms of the objects involved:

In [4]:
mpirun -f ${PBS_NODEFILE} -np 1 ./dense_matrix_vector_product -p rows
mpirun -f ${PBS_NODEFILE} -np 8 ./dense_matrix_vector_product -p rows
mpirun -f ${PBS_NODEFILE} -np 8 ./dense_matrix_vector_product -p cols

Right vector norm: 18.6581
Matrix Frobenius norm: 596.735
[0] Average time per matvec: 0.000966403
Left vector norm: 318.194
Right vector norm: 18.6581
Matrix Frobenius norm: 596.735
[0] Average time per matvec: 0.000149651
Left vector norm: 318.194
Right vector norm: 18.6581
Matrix Frobenius norm: 596.735
[0] Average time per matvec: 0.000128451
Left vector norm: 318.194


The 2D partitioned version is not written yet: as such, the matrix and left vector norms are wrong:

In [2]:
mpirun -f ${PBS_NODEFILE} -np 8 ./dense_matrix_vector_product -p 2d

Right vector norm: 18.6581
rank: 0
rank: 1
rank: 2
rank: 3
rank: 5
rank: 6
matrix_M: 768
 896
matrix_V: 896
 768
rank: 7
Matrix Frobenius norm: 0
matrix_M: 0
 128
matrix_V: 128
 0
matrix_M: 0
 128
matrix_V: 128
 0
matrix_M: 0
 128
matrix_V: 128
 0
matrix_M: 0
 128
matrix_V: 128
 0
matrix_M: 0
 128
matrix_M: 128
 256
matrix_V: 256
 128
matrix_M: 128
 256
matrix_V: 256
 128
matrix_M: 128
 256
matrix_V: 256
 128
matrix_M: 128
 256
matrix_V: 256
 128
matrix_M: 128
 256
matrix_V: 256
 128
matrix_M: 128
 256
matrix_V: 256
 128
matrix_M: 256
 384
matrix_V: 384
 256
matrix_M: 256
 384
matrix_V: 384
 256
matrix_M: 256
 384
matrix_V: 384
 256
matrix_M: 256
 384
matrix_V: 384
 256
matrix_M: 256
 384
matrix_V: 384
 256
matrix_M: 256
 384
matrix_V: 384
 256
matrix_M: 256
 384
matrix_V: 384
 256
matrix_M: 384
 512
matrix_V: 512
 384
matrix_M: 384
 512
matrix_V: 512
 384
matrix_M: 384
 512
matrix_V: 512
 384
matrix_M: 384
 512
matrix_V: 512
 384
matrix_M: 384
 512
matrix_V: 512
 384
matrix_M: 384
 51

To complete the code, you must write the following routines:

In [6]:
grep -B 3 -A 23 "MatrixGetLocalRange2d" dmv.c | pygmentize -l C

/* Given arguments (which include a communicator, args->comm),
 * offsets for each rank in the left and right vectors compatible with a matrix (lOffsets and rOffsets),
 * compute which entries in the matrix this MPI rank will own, given by the column ranges [mStart, mEnd) and row ranges [nStart, nEnd) */
int MatrixGetLocalRange2d(Args args, const int *lOffsets, const int *rOffsets, int *mStart_p, int *mEnd_p, int *nStart_p, int *nEnd_p)
{
  MPI_Comm comm = args->comm;
  int      mStart, mEnd, nStart, nEnd;
  int      size, rank;
  int      err;

  /* initialize to bogus values */
  mStart = mEnd = nStart = nEnd = -1;
  err = MPI_Comm_size(comm, &size); MPI_CHK(err);
  err = MPI_Comm_rank(comm, &rank); MPI_CHK(err);
  /* TODO: compute mStart, mEnd, nStart, and nEnd. HINT: use DMVCommGetRankCoordinates2D() to get the
   * number of block columns and rows used to partition the matrix, mBlock and nBlock.
   * The block row i should contain the same rows as are in the left vector for
   * r

Note that `lOffsets` and `rOffsets` are like the `displs` argument to [MPI_Gatherv]: (https://www.mpich.org/static/docs/latest/www3/MPI_Gatherv.html) they are arrays of length `size + 1` (where `size` is the size of the MPI Communicator).

To complete this routine, you should figure out how you are going to set up your 2D grid of MPI processes in the following routine:

In [8]:
grep -m 1 -B 5 -A 11 "DMVCommGetRankCoordinates2D" dmv.c | pygmentize -l C

/* Given the number of processes in the MPI communicator, compute a 2D
 * grid size (num_rows x num_cols) for the communicator and the coordinates of
 * the current rank (row, col).  Make sure to use a column major ordering,
 * so that rank 1 gets coordates (1, 0), not (0,1) */
int DMVCommGetRankCoordinates2D(MPI_Comm comm, int *num_rows_p, int *row_p, int *num_cols_p, int *col_p)
{
  int num_cols, num_rows, col, row;
  
  MPI_Cart_coords(comm, myProcID, 2, coords);
  printf("Process: %d\trowID: %d\tcolID: %d\n", myProcID, row_id, col_id);
  num_cols = num_rows = col = row = -1;
  /* TODO: HINT, lookup MPI_Dims_create() */
  *num_cols_p = num_cols;
  *num_rows_p = num_rows;
  *col_p = col;
  *row_p = row;


Once you've computed the range of entries for a process, the program will fill those entries.  The only other thing you need to do is the actual matvec:

In [8]:
pygmentize -l c dmv_matvec_2d.c

#include "dmv.h"
#include "dmv_impl.h"

int DenseMatVec_2dPartition(Args args, int mStart, int mEnd, int nStart, int nEnd, const double *matrixEntries, int rStart, int rEnd, const double *vecRightLocal, int lEnd, int lStart, double *vecLeftLocal)
{
  MPI_Comm comm = args->comm;
  int      size, rank, err;

  err = MPI_Comm_size(comm, &size); MPI_CHK(err);
  err = MPI_Comm_rank(comm, &rank); MPI_CHK(err);
  /* TODO: implement a matrix-vector multiplication on a 2d matrix partition */
  /* HINT:
   * 1. Use DMVCommGetRankCoordinates2D() to get the coordinates of the current rank in a 2d grid of MPI processes.
   * 2. Use the coordinates as colorings to split the communicator into row and column communicators.
   * 3. Use MPI_Allgatherv() on the column communicator so that every process has the correct entries to multiply against its local portion of the matrix ([nStart, nEnd]).
   *      Look at DenseMatVec_RowPartition_Allgatherv() in dmv_matvec_row.c for an example of using MPI_Algathe

In the code, `matrixEntries` is a row-oriented 2D array of size `(mEnd - mStart) x (nEnd - nStart)` that represents the submatrix `A[mStart:(mEnd - 1),nStart:(nEnd - 1)]`.  `vecRightLocal` is an array of size `rEnd - rStart` that represents the subvector `x[rStart:(rEnd-1)]`, and `vecLeftLocal` is an array of size `lEnd - lStart` that represents the subvector `b[lStart:(lEnd-1)]`.  You may want to look at the implementations in [dmv_matvec_row.c](./dmv_matvec_row.c) and [dmv_matvec_col.c](./dmv_matvec_col.c), which have the same arguments.

When your code is correct, we should be able to run your implementation on different random inputs and get the same results as the other implementations:

In [8]:
make clean
make dense_matrix_vector_product
mpirun -f ${PBS_NODEFILE} -np 8 ./dense_matrix_vector_product -p 2d

rm -f *.o dense_matrix_vector_product
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dense_matrix_vector_product.o dense_matrix_vector_product.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_args.o dmv_args.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv.o dmv.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_global_size.o dmv_global_size.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_layout.o dmv_layout.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_matvec_2d.o dmv_matvec_2d.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_matvec.o dmv_matvec.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv_matvec_col.o dmv_matvec_col.c
mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 

In [25]:
make dense_matrix_vector_product
for i in {1..3}; do
  seed=`date +%s`
  echo "==== Iteration $i, seed $seed ===="
  echo "== row decomposition =="
  mpirun -f ${PBS_NODEFILE} -np ${PBS_NP} ./dense_matrix_vector_product -p cols -e $seed
  echo "== column decomposition =="
  mpirun -f ${PBS_NODEFILE} -np ${PBS_NP} ./dense_matrix_vector_product -p rows -e $seed
  echo "== 2D decomposition =="
  mpirun -f ${PBS_NODEFILE} -np ${PBS_NP} ./dense_matrix_vector_product -p 2d   -e $seed
done

mpicc -I../../utils/Random123/include -I../../utils -g -Wall -O3 -std=c99 -c -o dmv.o dmv.c
mpicc -o dense_matrix_vector_product dense_matrix_vector_product.o dmv_args.o dmv.o dmv_global_size.o dmv_layout.o dmv_matvec_2d.o dmv_matvec.o dmv_matvec_col.o dmv_matvec_row.o dmv_offset.o -lm -lrt
==== Iteration 1, seed 1571557490 ====
== row decomposition ==
Right vector norm: 18.6065
Matrix Frobenius norm: 604.634
[0] Average time per matvec: 0.000150255
Left vector norm: 195.725
== column decomposition ==
Right vector norm: 18.6065
Matrix Frobenius norm: 604.634
[0] Average time per matvec: 0.000540876
Left vector norm: 195.725
== 2D decomposition ==
Right vector norm: 18.6065
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Process: 28
Proces

**(Runtime analysis, 2 pts):** Use the $\lambda + \mu m$ complexity model of a message and the complexities of collectives that we discussed in class and normalize the cost of a single flop to 1.  Estimate for the 2D decomposition:

- $T_{\text{comp}}$ the local computation time on each process for an $n \times n$ matrix with $p$ processes,
- $T_{\text{net}}(n \times n, p)$, the network communication time of the 2D decomposition,
- $S(n\times n, p) = T(n \times n, 1) / T(n \times n, p)$, the speedup, and
- $\max_p S(n\times n, p)$, the maximum speedup over all number of processes.

You may assume $\sqrt{p}$ and $p$ both divide $n$.  In addition to the complexities of collectives that we have discussed in class, you may assume that the complexity of [MPI_Reduce_scatter](https://www.mpich.org/static/docs/latest/www3/MPI_Reduce_scatter.html) the sums vectors of length $mp$ from each process and scatters them across the processes is $\lambda \log_2 p + 2 \mu mp.$